####  Importing books we like

In [10]:
liked_books = ["818056","22010842","16050719","18114797","893172",'20662423']


#### mapping book id and csv id

In [13]:
csv_book_mapping = {}

with open(r"C:\Users\91702\Downloads\book_id_map.csv") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

####   Creating a set of books which we liked and other users liked and gave a rating > 4

In [15]:
overlap_users = set()
with open(r"C:\Users\91702\Downloads\goodreads_interactions.csv") as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id,csv_id,_,rating,_ = line.split(",")
        
        if user_id in overlap_users:
            continue
        
        try:
            rating = int(rating)
        except ValueError:
            continue
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
            overlap_users.add(user_id)

####  Finding out the books read by other users with same taste

In [17]:
rec_lines=[]
with open(r"C:\Users\91702\Downloads\goodreads_interactions.csv") as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id,csv_id,_,rating,_ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id,book_id,rating])
            
        

In [19]:
len(rec_lines)

2122032

In [20]:
import pandas as pd

In [31]:
recs = pd.DataFrame(rec_lines,columns=["user_id","book_id","ratings"])
recs["book_id"] = recs["book_id"].astype(str)
recs["ratings"] = recs["ratings"].astype(int)

In [39]:
top_recs = recs["book_id"].value_counts().head(10)
top_recs = top_recs.index.values

In [34]:
book_titles = pd.read_json("book_titles.json")
book_titles["book_id"] = book_titles["book_id"].astype(str)

In [40]:
book_titles[book_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,mod_title
73489,7260188,"Mockingjay (The Hunger Games, #3)",1743362,https://www.goodreads.com/book/show/7260188-mo...,https://images.gr-assets.com/books/1358275419m...,mockingjay the hunger games 3
196714,818056,Harry Potter and the Deathly Hallows (Harry Po...,13938,https://www.goodreads.com/book/show/818056.Har...,https://images.gr-assets.com/books/1351958236m...,harry potter and the deathly hallows harry pot...
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
323031,41865,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
352861,49041,"New Moon (Twilight, #2)",1161751,https://www.goodreads.com/book/show/49041.New_...,https://images.gr-assets.com/books/1361039440m...,new moon twilight 2
720115,93124,Harry Potter and the Half-Blood Prince (Harry ...,21235,https://www.goodreads.com/book/show/93124.Harr...,https://images.gr-assets.com/books/1361297007m...,harry potter and the half blood prince harry p...
790927,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
833311,6148028,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,catching fire the hunger games 2
952424,428263,"Eclipse (Twilight, #3)",1146155,https://www.goodreads.com/book/show/428263.Ecl...,https://images.gr-assets.com/books/1361038355m...,eclipse twilight 3
1095301,11870085,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,the fault in our stars


In [ ]:
# YOU see all the above books are very popular,lets create a more personalised reccomendation system

In [49]:
all_recs = recs["book_id"].value_counts()

In [50]:
type(all_recs)

pandas.core.series.Series

In [51]:
all_recs = all_recs.to_frame().reset_index()

In [52]:
all_recs.columns = ["book_id","book_count"]

In [54]:
all_recs

,book_id,book_count
0,818056,2150
1,2767052,1574
2,41865,1571
3,11870085,1393
4,6148028,1285
...,...,...
499006,18603924,1
499007,23508302,1
499008,22665598,1
499009,22361921,1


In [55]:
all_recs = all_recs.merge(book_titles,how="inner",on="book_id")

In [56]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"]/all_recs["ratings"])

In [58]:
all_recs.sort_values("score",ascending=False)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
0,818056,2150,Harry Potter and the Deathly Hallows (Harry Po...,13938,https://www.goodreads.com/book/show/818056.Har...,https://images.gr-assets.com/books/1351958236m...,harry potter and the deathly hallows harry pot...,331.647295
1770,24909347,105,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,134.451220
3628,26856502,63,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,113.400000
5622,32333338,45,Save the Date,19,https://www.goodreads.com/book/show/32333338-s...,https://images.gr-assets.com/books/1510611507m...,save the date,106.578947
6256,36307629,41,"King of Scars (King of Scars, #1)",22,https://www.goodreads.com/book/show/36307629-k...,https://images.gr-assets.com/books/1506962795m...,king of scars king of scars 1,76.409091
...,...,...,...,...,...,...,...,...
246663,22067311,1,Gray Mountain,16879,https://www.goodreads.com/book/show/22067311-g...,https://images.gr-assets.com/books/1403704826m...,gray mountain,0.000059
443230,46656,1,Foundation and Chaos (Second Foundation Trilog...,21614,https://www.goodreads.com/book/show/46656.Foun...,https://s.gr-assets.com/assets/nophoto/book/11...,foundation and chaos second foundation trilogy 2,0.000046
369701,4452257,1,The Book of Basketball: The NBA According to T...,24887,https://www.goodreads.com/book/show/4452257-th...,https://images.gr-assets.com/books/1320424763m...,the book of basketball the nba according to th...,0.000040
407884,785092,1,Secrets of the Millionaire Mind: Mastering the...,26552,https://www.goodreads.com/book/show/785092.Sec...,https://s.gr-assets.com/assets/nophoto/book/11...,secrets of the millionaire mind mastering the ...,0.000038


In [61]:
popular_recs = all_recs[all_recs["book_count"] > 75].sort_values("score",ascending=False)

In [60]:
popular_recs

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
2779,36072,77,The 7 Habits of Highly Effective People: Power...,330726,https://www.goodreads.com/book/show/36072.The_...,https://images.gr-assets.com/books/1421842784m...,the 7 habits of highly effective people powerf...,0.017927
2804,19691,76,"The Hunt for Red October (Jack Ryan Universe, #4)",274053,https://www.goodreads.com/book/show/19691.The_...,https://images.gr-assets.com/books/1281995542m...,the hunt for red october jack ryan universe 4,0.021076
2273,3609760,89,Twilight: The Complete Illustrated Movie Compa...,291293,https://www.goodreads.com/book/show/3609760-tw...,https://images.gr-assets.com/books/1352539022m...,twilight the complete illustrated movie companion,0.027193
2213,480479,90,Under the Tuscan Sun,282391,https://www.goodreads.com/book/show/480479.Und...,https://images.gr-assets.com/books/1320524083m...,under the tuscan sun,0.028684
2341,14748,87,"Good in Bed (Cannie Shapiro, #1)",251303,https://www.goodreads.com/book/show/14748.Good...,https://images.gr-assets.com/books/1327936464m...,good in bed cannie shapiro 1,0.030119
...,...,...,...,...,...,...,...,...
240,893172,366,Percy Jackson and the Lightning Thief (Percy J...,2296,https://www.goodreads.com/book/show/893172.Per...,https://images.gr-assets.com/books/1327880928m...,percy jackson and the lightning thief percy ja...,58.343206
18,43509,1034,Harry Potter and the Goblet of Fire (Harry Pot...,16347,https://www.goodreads.com/book/show/43509.Harr...,https://images.gr-assets.com/books/1374678990m...,harry potter and the goblet of fire harry pott...,65.403805
6,93124,1193,Harry Potter and the Half-Blood Prince (Harry ...,21235,https://www.goodreads.com/book/show/93124.Harr...,https://images.gr-assets.com/books/1361297007m...,harry potter and the half blood prince harry p...,67.023734
1770,24909347,105,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,134.451220


In [66]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val,val)
def show_image(val):
    return '<img src="{}" width=50></img></a>'.format(val,val)


In [69]:
popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
1770,24909347,105,"Obsidio (The Illuminae Files, #3)",82,Goodreads,,obsidio the illuminae files 3,134.451220
6,93124,1193,"Harry Potter and the Half-Blood Prince (Harry Potter, #6)",21235,Goodreads,,harry potter and the half blood prince harry potter 6,67.023734
18,43509,1034,"Harry Potter and the Goblet of Fire (Harry Potter, #4)",16347,Goodreads,,harry potter and the goblet of fire harry potter 4,65.403805
14,224912,1076,"Harry Potter and the Chamber of Secrets (Harry Potter, #2)",22635,Goodreads,,harry potter and the chamber of secrets harry potter 2,51.149812
16,464164,1058,"Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",22794,Goodreads,,harry potter and the prisoner of azkaban harry potter 3,49.107835
1592,17699853,114,"Chain of Gold (The Last Hours, #1)",287,Goodreads,,chain of gold the last hours 1,45.282230
2427,17699859,84,"Chain of Thorns (The Last Hours, #3)",185,Goodreads,,chain of thorns the last hours 3,38.140541
12,72193,1086,"Harry Potter and the Philosopher's Stone (Harry Potter, #1)",31614,Goodreads,,harry potter and the philosopher s stone harry potter 1,37.306130
1249,13541056,136,"The Queen of Air and Darkness (The Dark Artifices, #3)",516,Goodreads,,the queen of air and darkness the dark artifices 3,35.844961
2192,25446343,91,"The Muse of Nightmares (Strange the Dreamer, #2)",232,Goodreads,,the muse of nightmares strange the dreamer 2,35.693966
